In [1]:
! poetry install

Installing dependencies from lock file

No dependencies to install or update

Installing the current project: src (1.0.0)Installing the current project: src (1.0.0)


In [2]:
from os import environ as env

In [3]:
env['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.5.1,org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1 pyspark-shell'

In [4]:
from pyspark.sql import SparkSession, DataFrame

In [5]:
spark = (
    SparkSession.builder
    .master('local[*]')
    .appName('Spark Structured Streaming')
    .getOrCreate()
)

:: loading settings :: url = jar:file:/home/vscode/.cache/pypoetry/virtualenvs/src-pt6m5yLH-py3.11/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/vscode/.ivy2/cache
The jars for the packages stored in: /home/vscode/.ivy2/jars
org.apache.spark#spark-streaming-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ad510d92-0008-42e8-a789-9d69ae0483b9;1.0
	confs: [default]
	found org.apache.spark#spark-streaming-kafka-0-10_2.12;3.5.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.1 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.spark#spark-sql-kafka-0-1

In [6]:
df: DataFrame = (
    spark.readStream 
    .format('kafka')
    .option('kafka.bootstrap.servers', 'kafka:9092')
    .option('subscribe', 'logs')
    .load()
)

In [8]:
from pyspark.sql.functions import split, col, decode

In [9]:
df = (
    df.selectExpr("CAST(value AS STRING)")
    .select(split(col("value"), "\|").alias("data"))
    .select(
      col("data").getItem(0).alias("ip"),
      col("data").getItem(1).alias("datetime"),
      col("data").getItem(2).alias("verb"),
      col("data").getItem(3).alias("uri"),
      col("data").getItem(4).alias("user_agent")
  )
)

In [11]:
df.printSchema()

root
 |-- ip: string (nullable = true)
 |-- datetime: string (nullable = true)
 |-- verb: string (nullable = true)
 |-- uri: string (nullable = true)
 |-- user_agent: string (nullable = true)



In [13]:
verb_counts = df.groupBy("verb").count()

query = (
  verb_counts.writeStream
  .outputMode("complete")
  .trigger(processingTime='10 seconds')
  .format("console")
  .start()
)
query.awaitTermination()

24/05/06 06:51:53 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-042c3248-8ee0-4054-bad8-bea95643a88a. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/05/06 06:51:53 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/05/06 06:51:53 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
24/05/06 06:52:04 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 10479 milliseconds


-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|verb|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+----+-----+
|verb|count|
+----+-----+
| GET|    1|
+----+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+----+-----+
|verb|count|
+----+-----+
|POST|    2|
| PUT|    1|
| GET|    4|
+----+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+------+-----+
|  verb|count|
+------+-----+
|  POST|    2|
|DELETE|    1|
|   PUT|    4|
|   GET|    5|
+------+-----+



...

-------------------------------------------
Batch: 28
-------------------------------------------
+------+-----+
|  verb|count|
+------+-----+
|  POST|   19|
|DELETE|    9|
|   PUT|   27|
|   GET|   79|
+------+-----+



-------------------------------------------
Batch: 29
-------------------------------------------
+------+-----+
|  verb|count|
+------+-----+
|  POST|   20|
|DELETE|    9|
|   PUT|   29|
|   GET|   82|
+------+-----+



ERROR:root:KeyboardInterrupt while sending command.              (72 + 8) / 200]
Traceback (most recent call last):
  File "/home/vscode/.cache/pypoetry/virtualenvs/src-pt6m5yLH-py3.11/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/vscode/.cache/pypoetry/virtualenvs/src-pt6m5yLH-py3.11/lib/python3.11/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

-------------------------------------------
Batch: 30
-------------------------------------------
+------+-----+
|  verb|count|
+------+-----+
|  POST|   20|
|DELETE|    9|
|   PUT|   29|
|   GET|   85|
+------+-----+

